Welcome to the Cars vs. Bikes Data Gatherer. Find where in your city biking is superior

In [ ]:
import json
import googlemaps
from datetime import datetime
gmaps = googlemaps.Client(key='YOUR_API_KEY')


The below block of code creates a grid of points within a geographic boundary which is inputted as a kml file Default: Island of Montreal

In [ ]:
# Generate a grid of points
import os
import fiona
fiona.drvsupport.supported_drivers['KML'] = 'rw'
import geopandas as gpd
from shapely.geometry import Point

# Load the KML file using geopandas
gdf = gpd.read_file('montreal_boundaries.kml', driver='KML')

# Find the largest polygon in the GeoDataFrame
largest_polygon = gdf['geometry'].iloc[gdf['geometry'].area.idxmax()]

# Create a list to store the generated points
points_within_polygon = []

# Define the number of points you want to generate along each axis
num_points_x = 32
num_points_y = 32

# Generate points in a grid within the bounding box of the largest polygon
minx, miny, maxx, maxy = largest_polygon.bounds
step_x = (maxx - minx) / (num_points_x - 1)
step_y = (maxy - miny) / (num_points_y - 1)

for i in range(num_points_x):
    for j in range(num_points_y):
        x = minx + i * step_x
        y = miny + j * step_y
        point = Point(x, y)
        if point.within(largest_polygon):
            points_within_polygon.append(point)

points_kml = 'points.kml'
# Check if the file exists before attempting to delete it
if os.path.exists(points_kml):
    # Delete the file
    os.remove(points_kml)
    print(f"'{points_kml}' has been deleted.")
else:
    print(f"'{points_kml}' does not exist.")

# Optionally, you can save the generated points as a GeoDataFrame
gdf_points = gpd.GeoDataFrame(geometry=points_within_polygon, crs=gdf.crs)
gdf_points.crs = 'EPSG:4326'

gdf_points.to_file(points_kml, driver='KML')
print(points_within_polygon)
print(len(points_within_polygon))

This is to convert a GeoJSON to a kml file. This is already done but may be useful for your data

In [ ]:
#Convert montreal GeoJSON to kml
import geopandas as gpd

# Load the GeoJSON file
geojson_file = 'limites-terrestres.geojson'
gdf = gpd.read_file(geojson_file)

# Specify the output KML file
kml_file = 'montreal_boundaries.kml'

# Save the GeoDataFrame to KML
gdf.to_file(kml_file, driver='KML')

print(f'Successfully converted {geojson_file} to {kml_file}')

This defines the function call_gmaps. The function takes two places, a time, and a mode of transit, and outputs the trip duration, distance, and parsed route data

In [ ]:
def call_gmaps(place1,place2,time,mode_of_transit):
    
   
    data = gmaps.directions(place1,
                                     place2,
                                     mode=mode_of_transit,
                                     departure_time=time)
    data=data[0]

    parsed_data = json.loads(json.dumps(data))
    print(parsed_data)

    duration_text = parsed_data['legs'][0]['duration']['text']
    duration_value = parsed_data['legs'][0]['duration']['value']

    # Print the duration
    print(f"Duration of the trip: {duration_text}")
    print(f"Duration value (in seconds): {duration_value}")

    # Access the "distance" for the entire trip
    total_distance_text = parsed_data['legs'][0]['distance']['text']
    total_distance_value = parsed_data['legs'][0]['distance']['value']

    # Print the total distance
    print(f"Total distance of the trip: {total_distance_text}")
    print(f"Total distance value (in meters): {total_distance_value}")
    
    return duration_value,total_distance_value,parsed_data



The below script is what calls the API to produce the data comparing biking and driving

In [ ]:
#Calculating distance to downtown not to everyother point
from varname import nameof
from datetime import datetime,timedelta
import pickle

num_points = len(points_within_polygon)
# Get the current date and time
current_datetime = datetime.now()

# Find the next Thursday
days_until_next_thursday = (3 - current_datetime.weekday() + 7) % 7
next_thursday = current_datetime + timedelta(days=days_until_next_thursday)

# Set the time to 4 PM
next_thursday_4pm = next_thursday.replace(hour=16, minute=0, second=0, microsecond=0)
time = next_thursday_4pm

w = [[0 for _ in range(4)] for _ in range(num_points)]
b = [[0 for _ in range(4)] for _ in range(num_points)]
d = [[0 for _ in range(4)] for _ in range(num_points)]
t = [[0 for _ in range(4)] for _ in range(num_points)]

mode_dict = {
    "w":"walking",
    "b":"bicycling",
    "d":"driving",
    "t": "transit"
}

#This function fills one of the nested lists above with data for each mode of transit
def fill_base2(abc,mode,num_points,points_within_polygon):
    for i in range(0,(num_points)):
        print(i)
        place1 = (points_within_polygon[i].y,points_within_polygon[i].x)
        #abc[i][0][0] = place1 
        place2 = (45.50191640802175, -73.56742138580896) #Centre of montreal
        abc[i][0] = place1
        abc[i][1] = place2
        abc[i][2:5] = call_gmaps(place1,place2,time,mode) #Note the number of times you call this as it costs you
        # for j in range(0,(num_points)):
        #     print(j)
        #     if j != i:
        #         place2 = (points_within_polygon[j].y,points_within_polygon[j].x) 
        #         abc[i][j][0] = place1
        #         abc[i][j][1] = place2
        #         if j < i:
        #             abc[i][j][2:5] = abc[j][i][2:5]
        #         else:
        #             abc[i][j][2:5] = call_gmaps(place1,place2,time,mode)
    return(abc)  

#Function is called and all data is saved      
mode = mode_dict[nameof(d)]
d = fill_base2(d,mode,num_points,points_within_polygon)
mode = mode_dict[nameof(b)]
b = fill_base2(b,mode,num_points,points_within_polygon)
# Save variable to a file
with open('b_data2.pkl', 'wb') as file:
    pickle.dump(b, file)
with open('d_data2.pkl', 'wb') as file:
    pickle.dump(d, file)



The below script converts the data calculated above to the data required to produce a heat map

In [ ]:
#get stats for pickle 2
num_points = len(points_within_polygon)
# import relevant packages
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
import csv
import os
import fiona
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

#Set variables
parking_time = 300 #This is the estimated time it takes to park in seconds downtown

# import relevant data
mtl_map = gdf
# Load the 'w' variable from the file
# with open('w_data.pkl', 'rb') as file:
#     loaded_w = pickle.load(file)
with open('d_data2.pkl', 'rb') as file:
    loaded_d = pickle.load(file)
with open('b_data2.pkl', 'rb') as file:
    loaded_b = pickle.load(file)

print(loaded_b[0:num_points][0:num_points][0:2])

def get_stats2(loaded_dat_a,loaded_dat_b,num_points):
    stats = [[0 for _ in range(0,2)] for _ in range(num_points)]
    #for i in range(0,num_points):
        #print(loaded_dat_a[0][i][0])
    print(stats)    

    for i in range(0,num_points):
        

        stats[i][0] = loaded_dat_a[i][0]
        mode_a = loaded_dat_a[i][2]
        print(mode_a)
        
        mode_b = loaded_dat_b[i][2]
        print(mode_b)
        
        stat =  (mode_a+parking_time)/mode_b
        
        stats[i][1] = stat
        print(stats)

    #stats = [list(row) for row in zip(*stats)] #transpose stats
    stats_dict = conv_dict(stats)
    return(stats_dict)

stats_dict = get_stats2(loaded_d,loaded_b,num_points)
print(stats_dict)

# Create a DataFrame from the data
df = pd.DataFrame(stats_dict)

# Create a folium map centered around Montreal
montreal_map = folium.Map(location=[45.5088, -73.554], zoom_start=12)

# Extract intensity values from the dictionary
intensity_values = stats_dict['intensity']

# Calculate the minimum and maximum intensity
min_intensity = min(intensity_values)
print(min_intensity)
max_intensity = max(intensity_values)
print(max_intensity)
# Define a custom gradient for the heatmap colors
gradient = {min_intensity: 'red', 0.5: 'yellow', 1.0: 'blue', max_intensity: 'green'}

# Create a HeatMap layer using the data
# Using zip to create rows
#heat_data = list(zip(data_dict['latitude'], data_dict['longitude'], data_dict['intensity']))
heat_data = [[point["latitude"], point["longitude"], point["intensity"]] for index, point in df.iterrows()]
print(heat_data)
HeatMap(heat_data, 
        radius = 5,
        min_opacity=0.4,
        blur = 3).add_to(montreal_map)

# Save the map to an HTML file
montreal_map.save("montreal_heatmap.html")


# Specify the CSV file path
csv_file_path = "heat_data.csv"

# Write the data to the CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    # Create a CSV writer object
    csv_writer = csv.writer(csvfile)

    # Write the header if needed
    # csv_writer.writerow(["latitude", "longitude", "intensity"])

    # Write the data
    csv_writer.writerows(heat_data)

print(f'Heat data has been saved to {csv_file_path}')

# This creates a visualization of all the points greater than 1 which can be plugged into google earth
# Create a GeoDataFrame from the DataFrame
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

# Filter points where the ratio is above 1
gdf_filtered = gdf[gdf['intensity'] > 1]

# Specify the output KML file
kml_file = 'filtered_points.kml'

# Save the filtered GeoDataFrame to KML
gdf_filtered.to_file(kml_file, driver='KML')


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
from shapely.geometry import Polygon

# Load the GeoJSON file containing road polygons
gdf_roads = gpd.read_file('roads_halved.geojson')

# Load the CSV file containing the grid of data points
csv_file = 'heat_data.csv'
df_grid = pd.read_csv(csv_file, header=None, names=['latitude', 'longitude', 'data_point'])

# Assuming you have columns 'longitude', 'latitude', and 'data_point' in the CSV
# If not, replace these column names with your actual column names
grid_coordinates = df_grid[['longitude', 'latitude']].values
grid_values = df_grid['data_point'].values

# Function to interpolate data for a polygon
def interpolate_data(polygon, grid_coordinates, grid_values):
    polygon_points = np.array(polygon.exterior.coords)
    interpolated_values = griddata(grid_coordinates, grid_values, polygon_points, method='linear')
    return np.nanmean(interpolated_values)  # Using nanmean to handle possible NaN values

# Create a new column 'interpolated_data' in the roads GeoDataFrame
gdf_roads['interpolated_data'] = gdf_roads['geometry'].apply(lambda polygon: interpolate_data(polygon, grid_coordinates, grid_values))

# Save the GeoDataFrame with interpolated data to a new GeoJSON file
gdf_roads.to_file('roads_with_interpolated_data2', driver='GeoJSON')

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.interpolate import griddata

# Load the GeoJSON file containing road polygons
gdf_roads = gpd.read_file('roads_halved.geojson')

# Load the CSV file containing the grid of data points
csv_file = 'heat_data.csv'
df_grid = pd.read_csv(csv_file, header=None, names=['latitude', 'longitude', 'data_point'])

# Assuming you have columns 'longitude', 'latitude', and 'data_point' in the CSV
# If not, replace these column names with your actual column names
grid_coordinates = df_grid[['longitude', 'latitude']].values
grid_values = df_grid['data_point'].values

# Function to interpolate data for a geometry
def interpolate_data(geometry, grid_coordinates, grid_values):
    if geometry.type == 'Polygon':
        polygon_points = np.array(geometry.exterior.coords)
    elif geometry.type == 'LineString':
        polygon_points = np.array(geometry.coords)
    else:
        raise ValueError("Unsupported geometry type")

    interpolated_values = griddata(grid_coordinates, grid_values, polygon_points, method='linear')
    return np.nanmean(interpolated_values)  # Using nanmean to handle possible NaN values

# Create a new column 'interpolated_data' in the roads GeoDataFrame
gdf_roads['interpolated_data'] = gdf_roads['geometry'].apply(lambda geom: interpolate_data(geom, grid_coordinates, grid_values))

# Save the GeoDataFrame with interpolated data to a new GeoJSON file
gdf_roads.to_file('roads_with_interpolated_data2.geojson', driver='GeoJSON')


This is an important function. Run it

In [ ]:


def conv_dict(stats):
    data = {
        "latitude": [],
        "longitude": [],
        "intensity": []
    }

    # Iterate through the stats list and populate the data dictionary
    for point, intensity in stats:
        latitude, longitude = point
        data["latitude"].append(latitude)
        data["longitude"].append(longitude)
        data["intensity"].append(intensity)

    # Print the resulting data dictionary
    return(data)

In [ ]:
import json
from shapely.geometry import LineString, Polygon, mapping

# Load your GeoJSON data
with open('roads_with_interpolated_data2.geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)

# Function to convert LineString to Polygon
def convert_line_to_polygon(line_string):
    coordinates = line_string['coordinates']
    # Close the LineString to create a ring
    coordinates.append(coordinates[0])
    polygon = Polygon(coordinates)
    return mapping(polygon)

# Iterate through features and convert LineString to Polygon
for feature in geojson_data['features']:
    if feature['geometry']['type'] == 'LineString':
        feature['geometry'] = convert_line_to_polygon(feature['geometry'])

# Save the modified GeoJSON with Polygons
with open('output_geojson_with_polygons.geojson', 'w') as output_geojson:
    json.dump(geojson_data, output_geojson, indent=2)

print("Conversion complete. GeoJSON with Polygons saved to output_geojson_with_polygons.geojson")


In [ ]:
import json
from shapely.geometry import LineString, Polygon, mapping

# Load your GeoJSON data
with open('roads_with_interpolated_data2.geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)

# Function to convert LineString to Polygon
def convert_line_to_polygon(line_string):
    coordinates = line_string['coordinates']
    # Close the LineString to create a ring
    coordinates.append(coordinates[0])
    polygon = Polygon(coordinates)
    return mapping(polygon)

# Iterate through features and convert LineString to Polygon
for feature in geojson_data['features']:
    if feature['geometry']['type'] == 'LineString':
        feature['geometry'] = convert_line_to_polygon(feature['geometry'])

# Save the modified GeoJSON with Polygons
with open('output_geojson_with_polygons.geojson', 'w') as output_geojson:
    json.dump(geojson_data, output_geojson, indent=2)

print("Conversion complete. GeoJSON with Polygons saved to output_geojson_with_polygons.geojson")


In [ ]:
import geopandas as gpd
from shapely.geometry import LineString

def simplify_geometry(geometry, tolerance=0.0001):
    if geometry.geom_type == 'LineString':
        simplified_geometry = LineString(geometry.simplify(tolerance))
        return simplified_geometry
    return geometry

def reduce_geojson_size(input_geojson, output_geojson, tolerance=0.0001):
    # Load the GeoJSON file
    gdf = gpd.read_file(input_geojson)

    # Apply simplification to each geometry in the GeoDataFrame
    gdf['geometry'] = gdf['geometry'].apply(lambda geom: simplify_geometry(geom, tolerance))

    # Save the simplified GeoDataFrame to a new GeoJSON file
    gdf.to_file(output_geojson, driver='GeoJSON')

    print(f"GeoJSON size reduced. Result saved to {output_geojson}")

# Example usage
input_geojson_file = "output_geojson_with_polygons.geojson"  # Replace with the path to your input GeoJSON
output_geojson_file = "roads_reduced_polygon.geojson"  # Replace with the desired output path

reduce_geojson_size(input_geojson_file, output_geojson_file, tolerance=0.001)


In [ ]:
import geopandas as gpd

def remove_properties(input_geojson, output_geojson):
    # Load GeoJSON file
    gdf = gpd.read_file(input_geojson)

    # Remove properties for each feature
    for index, row in gdf.iterrows():
        gdf.at[index, 'properties'] = {}

    # Save modified GeoJSON
    gdf.to_file(output_geojson, driver='GeoJSON')

    print(f'Properties removed and GeoJSON saved to {output_geojson}')

# Example usage:
input_geojson_file = r"C:\Users\steve\Cloud-Drive\Car free centre ville\geobase.json"
#input_geojson_file = "C:\Users\steve\Cloud-Drive\Car free centre ville\geobase.json"
output_geojson_file = r"C:\Users\steve\Cloud-Drive\Car free centre ville\roads_size_reduce.geojson"

remove_properties(input_geojson_file, output_geojson_file)

In [ ]:
import geopandas as gpd

def remove_properties(input_geojson, output_geojson):
    # Load GeoJSON file
    gdf = gpd.read_file(input_geojson)

    # Set an empty dictionary for the 'properties' column for all rows
    gdf['properties'] = [{} for _ in range(len(gdf))]

    # Save modified GeoJSON
    gdf.to_file(output_geojson, driver='GeoJSON')

    print(f'Properties removed and GeoJSON saved to {output_geojson}')

# Example usage:
input_geojson_file = r"C:\Users\steve\Cloud-Drive\Car free centre ville\geobase.json"
output_geojson_file = r"C:\Users\steve\Cloud-Drive\Car free centre ville\roads_size_reduce.geojson"

remove_properties(input_geojson_file, output_geojson_file)

In [ ]:
import geopandas as gpd

def remove_shapes_at_edges(input_geojson, output_geojson):
    # Load GeoJSON file
    gdf = gpd.read_file(input_geojson)

    # Create a buffer around the entire geometry
    buffer_distance = 0.01  # Adjust the buffer distance as needed
    buffered_geometry = gdf.unary_union.buffer(buffer_distance)

    # Remove shapes at the edges by taking the difference
    gdf = gdf.difference(buffered_geometry)

    # Save modified GeoJSON
    gdf.to_file(output_geojson, driver='GeoJSON')

    print(f'Shapes at edges removed, and GeoJSON saved to {output_geojson}')

# Example usage:
input_geojson_file = r"C:\Users\steve\Cloud-Drive\Car free centre ville\roads_size_reduce.geojson"
output_geojson_file = r"C:\Users\steve\Cloud-Drive\Car free centre ville\roads_no_edges.geojson"

remove_shapes_at_edges(input_geojson_file, output_geojson_file)


In [ ]:
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString

def cut_geojson_in_half(input_geojson, output_geojson):
    # Load GeoJSON file
    gdf = gpd.read_file(input_geojson)

    # Find the bounding box center
    minx, miny, maxx, maxy = gdf.total_bounds
    center_x, center_y = (minx + maxx) / 2, (miny + maxy) / 2

    # Create a vertical line through the center
    center_line = LineString([(center_x, miny), (center_x, maxy)])

    # Split geometries into two based on the center line
    split_geoms = []
    for geom in gdf['geometry']:
        if geom.intersects(center_line):
            split_geoms.extend(geom.difference(center_line).geoms)
        else:
            split_geoms.append(geom)

    # Create a new GeoDataFrame with the split geometries
    split_gdf = gpd.GeoDataFrame(geometry=split_geoms, crs=gdf.crs)

    # Save the split GeoJSON
    split_gdf.to_file(output_geojson, driver='GeoJSON')

    print(f'GeoJSON split in half and saved to {output_geojson}')

# Example usage:
input_geojson_file = r"C:\Users\steve\Cloud-Drive\Car free centre ville\roads_size_reduce.geojson"
output_geojson_file = r"C:\Users\steve\Cloud-Drive\Car free centre ville\roads_no_edges.geojson"

cut_geojson_in_half(input_geojson_file, output_geojson_file)


In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon
import pandas as pd

# Load the grid data (assuming no column names and columns in order: latitude, longitude, ratio)
grid_data = pd.read_csv('heat_data.csv', header=None, names=['latitude', 'longitude', 'ratio'])

# Assuming you have a GeoDataFrame for Montreal city boundaries
montreal_boundaries = gpd.read_file('limites-terrestres.geojson')

# Convert the DataFrame to a GeoDataFrame
geometry = [Polygon([(row[1], row[0]), (row[1] + 1, row[0]), (row[1] + 1, row[0] + 1), (row[1], row[0] + 1)]) for _, row in grid_data.iterrows()]
grid_data_gdf = gpd.GeoDataFrame(grid_data, geometry=geometry, crs='EPSG:4326')

# Set the GeoDataFrame's CRS
grid_data_gdf.crs = 'EPSG:4326'

# Filter grid data where the ratio is greater than one
filtered_grid_data = grid_data_gdf[grid_data_gdf['ratio'] > 1]

# Intersect the polygons with Montreal boundaries to clip the result within Montreal
result = gpd.overlay(filtered_grid_data, montreal_boundaries, how='intersection')


# Save the result to a GeoJSON file
result.to_file('biking_boundary.geojson', driver='GeoJSON')

print("Shape created for areas where the ratio is greater than one. Result saved to result.geojson.")
